In [9]:
# Sequence to sequence Encoder Decoder architecture using Keras 

In [1]:
# import necessary packages
import numpy as np
import tensorflow as tf
import re
import pandas as pd
from tensorflow.python.layers.core import Dense
import math
import gensim
import collections
import pandas as pd
from random import randint

C:\Users\amrapali\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed, Bidirectional
from keras.layers import LSTM
from keras.layers import Input, RepeatVector
from keras.models import Model
import keras.backend as K

Using TensorFlow backend.


In [3]:
# importing and pre-processing training data 
# Remove \n, "image*" ,"?/n" characters and add ? to show end of question. Separate words joined by underscore
# questData : contains only questions ending with "?"
# ansData : contains corresponding answersending with <End> tag
question_data = open("Text-Q&A-train.txt")
questData= []
ansData = []
for line in question_data.readlines():
    line = line.replace("?\n","?")
    line = re.sub(r"image\d+", " ", line)
    line = line.replace(","," ,")
    line = line.replace("."," . ")
    line = line.replace("_"," ")
    if line.endswith("\n") or line== 'chair':
        line = line.replace("\n","")
        line = line + " "+ "<End>"
        ansData.append(line.split(" "))
    else:
        line = line[0:-11]
        line+= " ?"
        questData.append(line.split(" "))

In [6]:
# Load the trained word2vec model
from gensim.models import KeyedVectors
wv = KeyedVectors.load("word2vec-train1.kv", mmap='r')
wv.init_sims(replace=True)
# create a dataframe to store indexed vocabulary
vocab_df = pd.DataFrame(list(wv.vocab.keys()))
vocab_df.columns = ["vocab"]
vocab_df["index"]=vocab_df.index

In [18]:
vocab_df.describe

<bound method NDFrame.describe of             vocab  index
0         <start>      0
1           <Pad>      1
2           <Unk>      2
3               .      3
4            what      4
5              is      5
6              on      6
7             the      7
8           right      8
9            side      9
10             of     10
11          black     11
12      telephone     12
13            and     13
14           left     14
15            red     15
16          chair     16
17              ?     17
18          <End>     18
19           desk     19
20             in     20
21          front     21
22          white     22
23           door     23
24           book     24
25              ,     25
26        scissor     26
27         papers     27
28           tape     28
29      dispenser     29
...           ...    ...
972         piled    972
973       portion    973
974        covers    974
975          onto    975
976           its    976
977           put    977
978          bot

In [17]:
wv['what']

array([ 0.01564246, -0.07906352,  0.12545845, -0.03322148, -0.16345668,
       -0.11994962, -0.02419726, -0.00845344, -0.21154463, -0.14758609,
       -0.02498465, -0.14640003,  0.0053083 ,  0.02290317,  0.06954977,
       -0.07913482,  0.06960129,  0.04610439, -0.0704703 , -0.05515545,
       -0.02392741,  0.07041181, -0.00826276,  0.07135066,  0.04693804,
        0.1018002 , -0.17475079,  0.03019459, -0.02577956, -0.09709105,
       -0.02497012,  0.01327998,  0.10954046,  0.09873245, -0.00099137,
        0.01690316, -0.02797894, -0.10065802,  0.08162827,  0.10283701,
       -0.08164266,  0.05938908, -0.04398534,  0.03993543,  0.02023008,
        0.02725374, -0.22515999, -0.09475694, -0.12520815, -0.10349441,
       -0.05151626,  0.04778115,  0.01923627,  0.18967837, -0.1569497 ,
       -0.13052537,  0.07478965, -0.00357565,  0.14696053, -0.06089628,
        0.06991453, -0.10334751,  0.01655884,  0.02142748,  0.23947594,
        0.2963602 , -0.03883243, -0.0625682 ,  0.09651279, -0.01

In [9]:
# calculate the max question length in training data. 
maxQuestLen = 0
for i,u in enumerate(questData):
    if maxQuestLen < np.asarray(u).shape[0]:
        maxQuestLen = np.asarray(u).shape[0]

In [10]:
# initializing variables. 
batchSize = 50
epoch = 50
vecSize = wv.vector_size
vocabSize = vocab_df.shape[0]
startIx =0

In [11]:
# Encoder Decoder model building 
input1 = Input(shape=(maxQuestLen,vecSize), dtype='float32', name='input1')
lstm_out = Bidirectional(LSTM(128),merge_mode='ave')(input1)  #output_shape: (batchSize,128)
input2 = RepeatVector(maxQuestLen)(lstm_out)                 #output_shape: (batchSize,maxQuestLen,128)
lstm_out2, hidden_state2, cell_state2 = LSTM(128, return_sequences = True,
                                             return_state=True)(input2)   #lstm_out2: (batchSize,maxQuestLen,128)

output2 = (TimeDistributed(Dense(vocabSize, activation="softmax")))(lstm_out2)  #output_shape:(batchSize,maxQuestLen,vocabSize)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [12]:
#compile model using cross entropy loss and adam optimizer. 
# Add sample weight mode as temporal to associate weights to words during training. This controls the effects due to stopwords 
modelTFiDFWeights = Model(input=input1, outputs=output2)
modelTFiDFWeights.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['categorical_accuracy'],sample_weight_mode="temporal")

C:\Users\amrapali\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ti..., inputs=Tensor("in...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [65]:
#Training model using batches of data. 
# convert each word to vector, get target vector as one hot encodings. 
#encInp_shape: (batchSize,maxQuestLen,vecSize), target_shape :(batchSize,maxQuestLen,vocabSize)
# This part also adds weights to each input sample based of the true prediction of the word. This is done by calculating Tf-iDF 
# at a batchlevel for each word. Thus frequent stopwords like "the" are given less weights while training. 
# Improvements (WIP): 1. traing model using train and validation data 
for e in range(epoch):
    for itr in range(int(math.ceil(np.asarray(questData).shape[0]/batchSize))):
        if (np.asarray(questData).shape[0] - (itr*batchSize) >= batchSize):
            startIx = (itr*batchSize)%(np.asarray(questData).shape[0])
            #startIx = 0
        else:
            break
        #encoder Inputs
        encInp = np.ones((batchSize,maxQuestLen,vecSize))*wv["<Pad>"]
        for i,wordList in enumerate(questData[startIx:startIx+batchSize]):
            for w,word in enumerate(wordList):
                encInp[i][w] = wv[word] 
        
        # target labels as one hot encoding over vocabulary 
        target = np.zeros((batchSize,maxQuestLen,vocabSize))
        target[:,:,vocab_df["index"].loc[vocab_df["vocab"]== "<Pad>"]] = 1
        for i,questList in enumerate(questData[startIx:startIx+batchSize]):
            for w,quest in enumerate(questList):
                target[i,w,vocab_df["index"].loc[vocab_df["vocab"]== "<Pad>"]] = 0
                target[i,w,vocab_df["index"].loc[vocab_df["vocab"]== quest]] = 1
                
        # calculating weights for true labels 
        # vocabFreq (DF) : No. of samples in a given batch that contains the word (indexed using vocab dataFrame)  
        vocabFreq=np.zeros((1,vocabSize),dtype=float)
        for i,label in enumerate(np.argmax(target,2)):
            for l in np.unique(label):
                vocabFreq[0,l] +=1
        
        # calculating word frequency in a question
        # wordFreqDict (TF): dictionary contaiing count of words in a question
        sampleWeights = np.ones((batchSize,maxQuestLen))
        for i,questList in enumerate(questData[startIx:startIx+batchSize]):
            wordFreqDict = collections.Counter(questList)
            for w,quest in enumerate(questList):
                if vocabFreq[0,vocab_df["index"].loc[vocab_df["vocab"]== quest]] != batchSize:
                    #sampleWeights : tf-idf
                    sampleWeights[i,w] += (wordFreqDict[quest]/len(questList))*np.log(batchSize/vocabFreq[0,vocab_df["index"].loc[vocab_df["vocab"]== quest]])
        # training the model.             
        modelTFiDFWeights.fit(encInp, target,epochs=1,shuffle=True,batch_size=batchSize,sample_weight=sampleWeights)
    print("training epoch "+str(e)) 
    

Epoch 1/1
50/50 [==============================] - 7s - loss: 1.9516 - categorical_accuracy: 0.6536
Epoch 1/1
50/50 [==============================] - 0s - loss: 1.9835 - categorical_accuracy: 0.6921
Epoch 1/1
50/50 [==============================] - 0s - loss: 1.4037 - categorical_accuracy: 0.7821
Epoch 1/1
50/50 [==============================] - 0s - loss: 1.4832 - categorical_accuracy: 0.7421
Epoch 1/1
50/50 [==============================] - 1s - loss: 1.5561 - categorical_accuracy: 0.7429
Epoch 1/1
50/50 [==============================] - 0s - loss: 1.7035 - categorical_accuracy: 0.7329
Epoch 1/1
50/50 [==============================] - 0s - loss: 1.4660 - categorical_accuracy: 0.7536
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.9945 - categorical_accuracy: 0.8043
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.8169 - categorical_accuracy: 0.8357
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.7240 - categorical_accuracy: 0.8579


50/50 [==============================] - 0s - loss: 1.2642 - categorical_accuracy: 0.8164
Epoch 1/1
50/50 [==============================] - 1s - loss: 0.9310 - categorical_accuracy: 0.8557
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.9933 - categorical_accuracy: 0.8357
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.9530 - categorical_accuracy: 0.8579
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.9951 - categorical_accuracy: 0.8586
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.9811 - categorical_accuracy: 0.8579
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.9641 - categorical_accuracy: 0.8321
Epoch 1/1
50/50 [==============================] - 0s - loss: 1.0502 - categorical_accuracy: 0.8393
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.7745 - categorical_accuracy: 0.8593
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.7748 - categorical_accuracy: 0.8500
Epoch 1/1


50/50 [==============================] - 1s - loss: 1.0530 - categorical_accuracy: 0.8086
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.8440 - categorical_accuracy: 0.8457
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.8010 - categorical_accuracy: 0.8507
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.8535 - categorical_accuracy: 0.8414
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.8929 - categorical_accuracy: 0.8364
Epoch 1/1
50/50 [==============================] - 0s - loss: 1.1577 - categorical_accuracy: 0.7964
Epoch 1/1
50/50 [==============================] - 0s - loss: 1.0964 - categorical_accuracy: 0.8136
Epoch 1/1
50/50 [==============================] - 1s - loss: 1.0585 - categorical_accuracy: 0.8043
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.6911 - categorical_accuracy: 0.8643
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5824 - categorical_accuracy: 0.8800
Epoch 1/1


50/50 [==============================] - 0s - loss: 1.8223 - categorical_accuracy: 0.7243
Epoch 1/1
50/50 [==============================] - 0s - loss: 1.4407 - categorical_accuracy: 0.7871
Epoch 1/1
50/50 [==============================] - 0s - loss: 1.3952 - categorical_accuracy: 0.7836
Epoch 1/1
50/50 [==============================] - 0s - loss: 1.4714 - categorical_accuracy: 0.7786
Epoch 1/1
50/50 [==============================] - 0s - loss: 1.7365 - categorical_accuracy: 0.7314
Epoch 1/1
50/50 [==============================] - 0s - loss: 1.4115 - categorical_accuracy: 0.7771
Epoch 1/1
50/50 [==============================] - 1s - loss: 0.8521 - categorical_accuracy: 0.8293
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.7563 - categorical_accuracy: 0.8464
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.6883 - categorical_accuracy: 0.8636
Epoch 1/1
50/50 [==============================] - 1s - loss: 0.7200 - categorical_accuracy: 0.8614
Epoch 1/1


50/50 [==============================] - 0s - loss: 0.5738 - categorical_accuracy: 0.8957
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4469 - categorical_accuracy: 0.9036
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4292 - categorical_accuracy: 0.9136
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3929 - categorical_accuracy: 0.9193
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.6999 - categorical_accuracy: 0.8971
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.9120 - categorical_accuracy: 0.8550
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.6798 - categorical_accuracy: 0.8936
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5716 - categorical_accuracy: 0.9064
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.7271 - categorical_accuracy: 0.8743
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5658 - categorical_accuracy: 0.8957
Epoch 1/1


...

50/50 [==============================] - 0s - loss: 0.5622 - categorical_accuracy: 0.9057
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4214 - categorical_accuracy: 0.9243
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5051 - categorical_accuracy: 0.9243
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4194 - categorical_accuracy: 0.9364
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3020 - categorical_accuracy: 0.9486
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3252 - categorical_accuracy: 0.9450
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3840 - categorical_accuracy: 0.9386
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.6060 - categorical_accuracy: 0.9050
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.6121 - categorical_accuracy: 0.9021
Epoch 1/1
50/50 [==============================] - 1s - loss: 0.7848 - categorical_accuracy: 0.8586
Epoch 1/1


50/50 [==============================] - 0s - loss: 0.4219 - categorical_accuracy: 0.9250
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4850 - categorical_accuracy: 0.8993
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5619 - categorical_accuracy: 0.8864
Epoch 1/1
50/50 [==============================] - 1s - loss: 0.4237 - categorical_accuracy: 0.9193
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5163 - categorical_accuracy: 0.9071
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3867 - categorical_accuracy: 0.9271
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5900 - categorical_accuracy: 0.8900
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4379 - categorical_accuracy: 0.9136
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5113 - categorical_accuracy: 0.9064
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5409 - categorical_accuracy: 0.9029
Epoch 1/1


50/50 [==============================] - 0s - loss: 0.5013 - categorical_accuracy: 0.9150
Epoch 1/1
50/50 [==============================] - 0s - loss: 1.1891 - categorical_accuracy: 0.7857
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.8076 - categorical_accuracy: 0.8536
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.6411 - categorical_accuracy: 0.8821
Epoch 1/1
50/50 [==============================] - 1s - loss: 0.6374 - categorical_accuracy: 0.8950
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.8610 - categorical_accuracy: 0.8536
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.7329 - categorical_accuracy: 0.8729
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4642 - categorical_accuracy: 0.9107
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4219 - categorical_accuracy: 0.9157
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4800 - categorical_accuracy: 0.9121
Epoch 1/1


50/50 [==============================] - 0s - loss: 0.4722 - categorical_accuracy: 0.9264
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3854 - categorical_accuracy: 0.9400
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.2742 - categorical_accuracy: 0.9507
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3005 - categorical_accuracy: 0.9464
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3553 - categorical_accuracy: 0.9393
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5663 - categorical_accuracy: 0.9093
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5744 - categorical_accuracy: 0.9014
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.7326 - categorical_accuracy: 0.8579
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5091 - categorical_accuracy: 0.9207
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.7740 - categorical_accuracy: 0.9021
Epoch 1/1


50/50 [==============================] - 0s - loss: 0.5536 - categorical_accuracy: 0.8907
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4232 - categorical_accuracy: 0.9157
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.7102 - categorical_accuracy: 0.8543
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3593 - categorical_accuracy: 0.9336
Epoch 1/1
50/50 [==============================] - 1s - loss: 0.7116 - categorical_accuracy: 0.8614
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4562 - categorical_accuracy: 0.9057
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5058 - categorical_accuracy: 0.9064
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5550 - categorical_accuracy: 0.8957
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3911 - categorical_accuracy: 0.9321
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3954 - categorical_accuracy: 0.9286
Epoch 1/1


In [124]:
#save model
model_json = modelTFiDFWeights.to_json()
with open("modelTFiDFWeights-2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
modelTFiDFWeights.save_weights("modelTFiDFWeights-2.h5")

In [125]:
#testing predictions on training dataset
testNum = 10 
for i in range(testNum):
    prediction = []
    textIx = randint(0, encInp.shape[0])
    modelPred = modelTFiDFWeights.predict(encInp[textIx][np.newaxis,:,:])
    print("Original Question:"+str(questData[startIx+textIx]))
    for p in np.argmax(modelPred,2)[0]:
        prediction.append(np.asarray(vocab_df["vocab"].ix[vocab_df["index"]==p])[0])
    print("Decoded Question:"+str(prediction))

Original Question:['what', 'is', 'to', 'the', 'right', 'of', 'the', 'photo', '?']
Decoded Question:['what', 'is', 'to', 'the', 'right', 'of', 'the', 'sofa', '?', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>']
Original Question:['what', 'is', 'the', 'colour', 'of', 'the', 'chair', 'seat', '?']
Decoded Question:['what', 'is', 'the', 'colour', 'of', 'the', 'the', 'sofa', '?', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>']
Original Question:['how', 'many', 'chairs', 'are', 'there', '?']
Decoded Question:['how', 'many', 'chairs', 'are', 'there', '?', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>']
Or

In [100]:
# Testing on unseen data
# create test and validation data 
# separate test and validation samples 
validationIxs = range(0,3000)

# read data as validation and test sets separetly 
test_data = open("Text-Q&A-test.txt")
testDataList =[]
valDataList = []
for i,line in enumerate(test_data.readlines()):
    line = line.replace("?\n","?")
    line = re.sub(r"image\d+", " ", line)
    line = line.replace(","," ,")
    line = line.replace("."," . ")
    line = line.replace("_"," ")
    if line.endswith("\n") or line== 'chair':
        continue
    else:
        line = line[0:-11]
        line+= " ?"
        if i in validationIxs:
            valDataList.append(line.split(" "))
        else:
            testDataList.append(line.split(" "))
        

# get word vectors for test and validation 
testInput = np.ones((len(testDataList),maxQuestLen,vecSize))*wv["<Pad>"]
valInput = np.ones((len(valDataList),maxQuestLen,vecSize))*wv["<Pad>"]
for i,testQuestList in enumerate(testDataList):
    for w,testQuest in enumerate(testQuestList):
        if testQuest not in wv.vocab:
            testInput[i][w] = wv["<Unk>"]
        else:
            testInput[i][w] = wv[testQuest] 
            
for i,valQuestList in enumerate(valDataList):
    for w,valQuest in enumerate(valQuestList):
        if valQuest not in wv.vocab:
            valInput[i][w] = wv["<Unk>"]
        else:
            valInput[i][w] = wv[valQuest] 
            
#validation target
valTarget = np.zeros((len(valDataList),maxQuestLen,vocabSize))
valTarget[:,:,vocab_df["index"].loc[vocab_df["vocab"]== "<Pad>"]] = 1
for i,valquestList in enumerate(valDataList):
    for w,valQuest in enumerate(valquestList):
        valTarget[i,w,vocab_df["index"].loc[vocab_df["vocab"]== "<Pad>"]] = 0
        if valQuest not in wv.vocab:
            valTarget[i,w,vocab_df["index"].loc[vocab_df["vocab"]== "<Unk>"]] = 1
        else:
            valTarget[i,w,vocab_df["index"].loc[vocab_df["vocab"]== valQuest]] = 1

In [123]:
#testing predictions on test dataset
testNum = 10 
for i in range(testNum):
    prediction = []
    textIx = randint(0, testInput.shape[0])
    modelPred = modelTFiDFWeights.predict(testInput[textIx][np.newaxis,:,:])
    print("Original Question:"+str(testDataList[textIx]))
    for p in np.argmax(modelPred,2)[0]:
        prediction.append(np.asarray(vocab_df["vocab"].ix[vocab_df["index"]==p])[0])
    print("Decoded Question:"+str(prediction))
        

Original Question:['what', 'is', 'the', 'colour', 'of', 'the', 'chair', '?']
Decoded Question:['what', 'is', 'the', 'colour', 'of', 'the', 'sofa', '?', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>']
Original Question:['what', 'is', 'on', 'the', 'door', '?']
Decoded Question:['what', 'is', 'on', 'the', 'wall', '?', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>']
Original Question:['how', 'many', 'screens', 'are', '?']
Decoded Question:['how', 'many', 'chairs', 'are', '?', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>']
Original Question:['what', 'is', '